In [34]:
!pip install ipywidgets

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from scipy.stats import norm

In [38]:
def generate_price_path(S0, mu, sigma, T, steps):
    """
    Generates a stock price path using Geometric Brownian Motion.

    Args:
        S0 (float): Initial stock price.
        mu (float): Expected annual return (drift).
        sigma (float): Annual volatility.
        T (int): Time to expiration in years.
        steps (int): Number of time steps.

    Returns:
        numpy.ndarray: An array of stock prices for the path.
    """
    dt = T / steps
    Z = np.random.standard_normal(steps)
    price_path = S0 * np.exp(np.cumsum((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z))
    return np.insert(price_path, 0, S0)


def black_scholes_price(S0, K, T, r, sigma):
    """
    Calculates the Black-Scholes price for a European call and put option.
    
    Args:
        S0 (float): Current stock price.
        K (float): Strike price.
        T (float): Time to maturity in years.
        r (float): Risk-free interest rate.
        sigma (float): Volatility.
        
    Returns:
        tuple: (call_price, put_price)
    """
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    call_price = (S0 * norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * norm.cdf(d2, 0.0, 1.0))
    put_price = (K * np.exp(-r * T) * norm.cdf(-d2, 0.0, 1.0) - S0 * norm.cdf(-d1, 0.0, 1.0))
    
    return call_price, put_price


def run_monte_carlo_simulation(S0, strike_price, T, mu, sigma, r, steps, num_simulations):
    """
    Prices options and visualizes the simulation.
    
    Args:
        S0 (float): Initial stock price.
        strike_price (float): Option's strike price.
        T (float): Time to expiration in years.
        mu (float): Expected annual return (drift).
        sigma (float): Annual volatility.
        r (float): Annual risk-free interest rate.
        steps (int): Number of time steps.
        num_simulations (int): Number of paths to generate.
    """
    
    final_prices = []
    call_payoffs = []
    put_payoffs = []
    running_avg_price = []
    
    # --- Run the Simulation ---
    for _ in range(num_simulations):
        path = generate_price_path(S0, mu, sigma, T, steps)
        final_prices.append(path[-1])
        
        # Calculate payoffs for this one path
        call_payoffs.append(max(0, path[-1] - strike_price))
        put_payoffs.append(max(0, strike_price - path[-1]))

        #Calculate the average of all payoffs so far
        current_avg_payoff = np.mean(call_payoffs)
        discounted_price = current_avg_payoff * np.exp(-r * T)
        running_avg_price.append(discounted_price)
        
    # --- Calculate Option Prices ---
    # We take the average payoff and "discount" it back to today's value
    # The discounting formula is: Present Value = Future Value * exp(-r * T)
    
    call_price_mc = np.mean(call_payoffs) * np.exp(-r * T)
    put_price_mc = np.mean(put_payoffs) * np.exp(-r * T)

    # ---PLOT: Convergence ---
    plt.figure(figsize=(12, 7))
    plt.plot(running_avg_price, color='lime')

    # --- Calculate Option Prices (Black-Scholes) ---
    call_price_bs, put_price_bs = black_scholes_price(S0, strike_price, T, r, sigma)
    
    print(f"--- Monte Carlo Results ({num_simulations:,} sims) ---")
    print(f"Call Price: ${call_price_mc:.2f}")
    print(f"Put Price:  ${put_price_mc:.2f}")
    print("\n--- Black-Scholes Exact Formula ---")
    print(f"Call Price: ${call_price_bs:.2f}")
    print(f"Put Price:  ${put_price_bs:.2f}")
    print("\n--- Difference ---")
    print(f"Call Diff: ${call_price_mc - call_price_bs:.4f}")
    print(f"Put Diff:  ${put_price_mc - put_price_bs:.4f}")
    
    # --- Plotting ---
    plt.style.use('dark_background')
    plt.figure(figsize=(12, 7))
    
    # Plot a subset of paths to keep the viz clean
    for i in range(min(num_simulations, 1000)):
        path = generate_price_path(S0, mu, sigma, T, steps)
        plt.plot(path, color='cyan', alpha=0.05)

    plt.axhline(y=strike_price, color='lime', linestyle='--', label=f'Strike Price: ${strike_price}')
    plt.title(f'{num_simulations} Simulated Price Paths')
    plt.xlabel(f'Steps ({steps} total)')
    plt.ylabel('Stock Price ($)')
    plt.legend()
    plt.grid(True, alpha=0.2)
    plt.show()

# --- Create the Interactive Widget ---
interact(run_monte_carlo_simulation,
         S0=IntSlider(value=100, min=50, max=200, step=1, description="Start Price:"),
         strike_price=IntSlider(value=110, min=50, max=200, step=1, description="Strike Price:"),
         T=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description="Years (T):"),
         mu=FloatSlider(value=0.05, min=-0.1, max=0.2, step=0.01, description="Drift (μ):"),
         sigma=FloatSlider(value=0.2, min=0.05, max=1.0, step=0.01, description="Volatility (σ):"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.005, description="Risk-Free Rate:"),
         steps=IntSlider(value=252, min=50, max=500, step=1, description="Steps:"),
         num_simulations=IntSlider(value=2000, min=500, max=10000, step=500, description="Simulations:")
);

interactive(children=(IntSlider(value=100, description='Start Price:', max=200, min=50), IntSlider(value=110, …